In [ ]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 9.2 MB/s eta 0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import os

ROOT_DIR = os.path.abspath('')
PRETRAINED_MODEL = '/content/drive/MyDrive/kuy/kuy/src_1/epoch=18-step=24947.ckpt'

In [ ]:
import gradio as gr

def image_classifier(inp):
    return {'ak': 0.3, 'bcc': 0.7}

demo = gr.Interface(fn=image_classifier, inputs="image", outputs="label")
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3cf05c9ac9d9cccbc3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
!!pip install torchmetrics
!pip install lightning
!pip install pytorch-lightning
!!pip install timm
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.9/693.9 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.7/727.7 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 11.3 MB/s eta 0:00:00


In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/kuy/kuy")
import torch
from src_1.models import ResNet18, BasicBlock

model = ResNet18(3, 7)

checkpoint = torch.load(PRETRAINED_MODEL)

# The state dict will contains net.layer_name
# Our model doesn't contains `net.` so we have to rename it
state_dict = checkpoint['state_dict']
for key in list(state_dict.keys()):
    if 'net.' in key:
        state_dict[key.replace('net.', '')] = state_dict[key]
        del state_dict[key]

model.load_state_dict(state_dict)
model.eval()

class_names = ['ak', 'bcc', 'bkl', 'bkl', 'df', 'mel', 'nv', 'vasc']
class_names.sort()

In [ ]:
import gradio as gr

import numpy as np
import cv2
from src_1.dataset import RGB_MEAN, RGB_STD, INPUT_SIZE
from torchvision.transforms import transforms

transformation_pipeline = transforms.Compose([
    transforms.ToPILImage(),
    transforms.CenterCrop(INPUT_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=RGB_MEAN, std=RGB_STD)
])


def preprocess_image(image: np.ndarray):
    """Preprocess the input image.

    Note that the input image is in RGB mode.

    Parameters
    ----------
    image: np.ndarray
        Input image from callback.
    """

    image = transformation_pipeline(image)
    image = torch.unsqueeze(image, 0)

    return image


def image_classifier(inp):
    """Image Classifier Function.

    Parameters
    ----------
    inp: Optional[np.ndarray] = None
        Input image from callback

    Returns
    -------
    Dict
        A dictionary class names and its probability
    """

    # If input not valid, return dummy data or raise error
    if inp is None:
        return {'ak': 0.3, 'bcc': 0.7}

    # preprocess
    image = preprocess_image(inp)
    image = image.to(dtype=torch.float32)

    # inference
    result = model(image)

    # postprocess
    result = torch.nn.functional.softmax(result, dim=1) # apply softmax
    result = result[0].detach().numpy().tolist() # take the first batch
    labeled_result = {name:score for name, score in zip(class_names, result)}

    return labeled_result

demo = gr.Interface(fn=image_classifier, inputs="image", outputs="label")
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ab025c073691f1bb16.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import os
sample_files = os.listdir('/content/drive/MyDrive/simple')
sample_files = [os.path.join('/content/drive/MyDrive/simple', path) for path in sample_files]

In [ ]:
import gradio as gr
def update(name):
    return f"Welcome to Gradio, {name}!"

with gr.Blocks() as demo:
    with gr.Row(): # Build a row
        with gr.Column(): # build a column section as the first item
            inp = gr.Image(label="image", image_mode="RGB") # build an image as the first column item
            with gr.Row(): # build a row section as the second item
                clear_btn = gr.Button("Clear")
                submit_btn = gr.Button("Submit")

        # build a label as the second item
        out = gr.Label(label="prediction", num_top_classes=3)

        # Define buttons functionalities
        submit_btn.click(fn=image_classifier, inputs=inp, outputs=out)
        clear_btn.click(
            lambda: (
                gr.update(value=None),
                gr.update(value=None),
            ),
            inputs=None,
            outputs=[inp, out]
        )

    # Add examples
    gr.Markdown("## Image Examples")
    gr.Examples(sample_files, inputs=[inp], label="Image Examples")

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://833d5cb6a775e94579.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
